In [2]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Warning and Error only

import tensorflow as tf
from tensorflow.python.client import device_lib
import re

print(f'Tensorflow v{tf.__version__}'.center(90, '·'))

tf_exp_config = tf.config.experimental

print('Local Devices:')
gpu_name_re = re.compile('name: ?([A-Za-z0-9 ]+)(?=(?:,|$))')
cp_cap_re = re.compile('compute capability: ?([0-9.]+)(?=(?:,|$))')
unit = ('M', 'G', 'T', 'P''E',)
for i, x in enumerate(device_lib.list_local_devices()):
    print(f'\t{i}: name \t= [{x.name}]')
    try:
        _desc = x.physical_device_desc
        print(f'\t{i}: model\t= [{gpu_name_re.search(_desc).group(1)}]')
        print(f'\t{i}: capablt.\t= [{cp_cap_re.search(_desc).group(1)}]')
        del _desc
    except AttributeError:
        ...
    print(f'\t{i}: type \t= [{x.device_type}]')
    _mem = x.memory_limit / 1024 / 1024
    unit_i = 0
    while _mem > 1024 and unit_i < len(unit):
        _mem /= 1024
        unit_i += 1
    print(f'\t{i}: memory\t= [{_mem:.1f} {unit[unit_i]}iB]')
    print()
    del _mem, unit_i
del gpu_name_re

print('Visible Devices:')
for i, x in enumerate(tf_exp_config.get_visible_devices()):
    print(f'\t{i}: {x}')
print()

print(f'Device Policy: {tf_exp_config.get_device_policy()}')
print()

print(f'Visible GPU Configs:')
gpus = tf_exp_config.get_visible_devices('GPU')
for i, x in enumerate(gpus):
    print(f'\t{i}: {x}')
    tf_exp_config.set_memory_growth(x, True)
    print(f'\t{i}: Memory Growth: {tf_exp_config.get_memory_growth(x)}')
print()

del tf_exp_config, unit

······························Tensorflow v2.2.0-dev20200112·······························
Local Devices:
	0: name 	= [/device:CPU:0]
	0: type 	= [CPU]
	0: memory	= [256.0 MiB]

	1: name 	= [/device:GPU:0]
	1: model	= [GeForce GTX 1080]
	1: capablt.	= [6.1]
	1: type 	= [GPU]
	1: memory	= [6.2 GiB]

Visible Devices:
	0: PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
	1: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

Device Policy: silent

Visible GPU Configs:
	0: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
	0: Memory Growth: True



In [3]:
from tensorflow.keras.mixed_precision import experimental as mixed_precision

for i, x in enumerate(device_lib.list_local_devices()):
    if x.device_type != 'GPU':
        continue
    _desc = x.physical_device_desc
    _capability = float(cp_cap_re.search(_desc).group(1))
    if _capability > 7:
        print('Enabling Mixed Precision'.center(90, '·'))
        float16_policy = mixed_precision.Policy('mixed_float16')
        mixed_precision.set_policy(float16_policy)
        print(f'\tCompute  dtype: {float16_policy.compute_dtype}')
        print(f'\tVariable dtype: {float16_policy.variable_dtype}')
        del float16_policy
    else:
        print(f'Skipping Mixed Precision due to compute capability [{_capability}] < 7')
    del cp_cap_re, _desc, _capability
    print()
    break
del i, x

Skipping Mixed Precision due to compute capability [6.1] < 7



In [4]:
tf.config.optimizer.set_jit(True)
print('Enabled XLA for TensorFlow models'.center(90, '·'))

#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#

····························Enabled XLA for TensorFlow models·····························


In [5]:
# Download Dataset
import numpy as np

keras = tf.keras

fashion_mnist = keras.datasets.fashion_mnist
(train_imgs, train_lbls), (test_imgs, test_lbls) = fashion_mnist.load_data()
train_imgs.shape = (train_imgs.shape[0], train_imgs.shape[1], train_imgs.shape[2], 1)
test_imgs.shape = (test_imgs.shape[0], test_imgs.shape[1], test_imgs.shape[2], 1)
labels = [
    '0 T-shirt/top',
    '1 Trouser',
    '2 Pullover',
    '3 Dress',
    '4 Coat',
    '5 Sandal',
    '6 Shirt',
    '7 Sneaker',
    '8 Bag',
    '9 Ankle boot'
]
print(labels)

['0 T-shirt/top', '1 Trouser', '2 Pullover', '3 Dress', '4 Coat', '5 Sandal', '6 Shirt', '7 Sneaker', '8 Bag', '9 Ankle boot']


In [6]:
# Normalize
if (
        (train_imgs.max()>1 and test_imgs.max()>1) or
        input(f'WARNING! Max value of train img or test img is less than 1: '
              f'[{train_imgs.max()}][{test_imgs.max()}]\n '
              f'Continue? [y/N]:').upper()=='Y'
):
    train_imgs = train_imgs / np.iinfo(train_imgs.dtype).max
    test_imgs = test_imgs / np.iinfo(test_imgs.dtype).max
    print(train_imgs[0][14])
    print(type(train_imgs[0]))
    print(train_imgs.min(), train_imgs.max())
    print('Data Normalization'.center(90, '·'))

[[0.        ]
 [0.        ]
 [0.00392157]
 [0.01568627]
 [0.02352941]
 [0.02745098]
 [0.00784314]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.92941176]
 [0.88627451]
 [0.85098039]
 [0.8745098 ]
 [0.87058824]
 [0.85882353]
 [0.87058824]
 [0.86666667]
 [0.84705882]
 [0.8745098 ]
 [0.89803922]
 [0.84313725]
 [0.85490196]
 [1.        ]
 [0.30196078]
 [0.        ]]
<class 'numpy.ndarray'>
0.0 1.0
····································Data Normalization····································


In [16]:
# Build Models
import os

if os.path.isdir('models/mnist_model/'):
    model = keras.models.load_model('models/mnist_model')
    print('Model Loaded'.center(90, '·'))

else:
    keras.backend.clear_session()
    model = keras.Sequential((
        keras.layers.Conv2D(filters=64,
                            kernel_size=5, 
                            input_shape=(28, 28, 1)),
        keras.layers.Activation(keras.activations.elu, 
                                dtype=tf.dtypes.float32),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPool2D(pool_size=2,strides=2),
        
        
        keras.layers.Conv2D(filters=128,
                            kernel_size=3),
        keras.layers.Activation(keras.activations.elu, 
                                dtype=tf.dtypes.float32),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPool2D(pool_size=2,strides=2),


        keras.layers.Flatten(),
        keras.layers.Dense(1024),
        keras.layers.Dropout(.1),
        
        keras.layers.Dense(10),
        keras.layers.Activation(keras.activations.softmax, 
                                dtype=tf.dtypes.float32)
    ))

model_cb =keras.callbacks.ModelCheckpoint(
    'models/mnist_model/',
    monitor='val_acc',
    save_best_only=True,
    verbose=True
)
stop_cb=keras.callbacks.EarlyStopping(
    patience=1,
    verbose=True
)

In [27]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.002),
    loss=keras.losses.sparse_categorical_crossentropy,
    metrics=['mae','acc']
)

model.summary()
print('Model Compiled'.center(90, '·'))

# losses = tf.nn.sigmoid_cross_entropy_with_logits(labels=self.input_y, logits=self.logits)
# losses = tf.reduce_mean(tf.reduce_sum(losses, axis=1), name="sigmoid_losses")
# l2_losses = tf.add_n([tf.nn.l2_loss(tf.cast(v, tf.float32)) for v in tf.trainable_variables()],
#                      name="l2_losses") * l2_reg_lambda
# self.loss = tf.add(losses, l2_losses, name="loss")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 64)        1664      
_________________________________________________________________
activation (Activation)      (None, 24, 24, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 24, 24, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 128)       73856     
_________________________________________________________________
activation_1 (Activation)    (None, 10, 10, 128)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 10, 128)       5

In [30]:
# Fit Model
from IPython.display import clear_output

show = 0
while not os.path.isfile('models/stop') and not os.path.isfile('models/stop.txt'):
    history:keras.callbacks.History = \
        model.fit(train_imgs, train_lbls,
              validation_split=.02,
              batch_size=128,
              callbacks=[model_cb, stop_cb])
    show = (show+1)%5
    if not show:
        clear_output()
    
print('Model Fit'.center(90, '·'))

Train on 58800 samples, validate on 1200 samples
58496/58800 [============================>.] - ETA: 0s - loss: 0.0261 - mae: 4.4203 - acc: 0.9961

In [ ]:
# Eval Test Result
test_loss, test_acc = model.evaluate(test_imgs, test_lbls)
print(test_loss, test_acc)

In [ ]:
# Prediction
import numpy as np

predict = model.predict(test_imgs)
np.argmax(predict[0])
# print(tf.argmax(predict[0]).numpy())
# print(test_lbls[0])